# 1. Load the dataset

### [KOR]  

데이터셋은 한국의 응급의료 분야에서 사용되는 KTAS (Korean Triage and Acuity Scale) 분류 데이터를 기반으로 합니다. 이 데이터셋은 보건복지부의 제 2015-243호 "한국 응급환자 중증도 분류기준"에 따라 구성되어 있으며, 총 4,368개의 분류 항목으로 이루어져 있습니다. 

KTAS는 환자가 호소하는 증상을 근거로 1, 2차 고려사항을 적용하여 환자의 응급도를 5단계로 나누어 평가하도록 구성되어 있는데, 각 데이터 항목은 다음과 같은 정보로 구성되어 있습니다.
- KTAS1_Group: 환자의 나이를 기준으로 15세를 기준으로 성인(A)과 소아(B)로 구분됩니다. 
- KTAS2_Group: 총 165가지의 주증상의 특성에 따라 공통으로 17개의 군으로 분류
- KTAS3_Group: 1차 고려사항은 대부분의 증상에서 공통으로 적용할 수 있는 특성을 가진 의식, 생체징후를 기반으로 한 혈역학적 상태, 호흡곤란의 정도, 발열, 통증, 출혈성 질환여부, 사고기전으로 구성되어 있다
- KTAS4_Group: 2차 고려사항은 특정한 증상에서 적용해야 하는 특성으로 구성되어 있다
- KTAS_Code: 각 분류 항목의 고유 코드입니다. 
- KTAS_Level: 환자의 응급 상태를 1~5의 척도로 나타내는 지표로, 1이 가장 긴급하고 5가 상대적으로 덜 긴급한 상황을 의미합니다.


### [ENG]
The dataset is based on the KTAS (Korean Triage and Acuity Scale) classification, which is utilized in the emergency medical field in Korea. This dataset is structured according to the Ministry of Health and Welfare's 2015-243 "Korean Emergency Patient Severity Classification Standard" and comprises a total of 4,368 classification items.

KTAS is structured to evaluate a patient's urgency level in five stages based on the symptoms the patient presents, applying both primary and secondary considerations. Each data entry consists of the following information:

- KTAS1_Group: Patients are categorized as either adult (A) or pediatric (B) based on the age criterion of 15 years.
- KTAS2_Group: This is based on the characteristics of the total 165 main symptoms, commonly grouped into 17 clusters.
- KTAS3_Group: The primary considerations mainly apply to a wide range of symptoms and consist of consciousness, physiological signs based on hemodynamic status, degree of respiratory distress, fever, pain, presence of hemorrhagic diseases, and mechanism of injury.
- KTAS4_Group: The secondary considerations consist of characteristics that need to be applied for specific symptoms.
- KTAS_Code: This is a unique code for each classification item.
- KTAS_Level: This is an indicator representing the urgency of a patient's condition on a scale from 1 to 5, with 1 being the most urgent and 5 being relatively less urgent.


In [2]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding

In [3]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [4]:
# load & inspect dataset
input_datapath = "../data/KTAS_DATA.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0, encoding='cp949')

df["Combined"] = df.KTAS2_Group.str.strip() + " - " + df.KTAS3_Group.str.strip() + " - " + df.KTAS4_Group.str.strip()
encoding = tiktoken.get_encoding(embedding_encoding)
df["n_tokens"] = df.Combined.apply(lambda x: len(encoding.encode(x)))
df.head()

,KTAS1_Group,KTAS1_Code,KTAS2_Group,KTAS2_Code,KTAS3_Group,KTAS3_Code,KTAS4_Group,KTAS4_Code,KTAS_CODE,KTAS_Level,Combined,n_tokens
Index,,,,,,,,,,,,
1,성인,A,물질오용,A,물질오용/중독,A,중증 호흡곤란,AA,AAAAA,1,물질오용 - 물질오용/중독 - 중증 호흡곤란,32
2,성인,A,물질오용,A,물질오용/중독,A,쇼크,AB,AAAAB,1,물질오용 - 물질오용/중독 - 쇼크,25
3,성인,A,물질오용,A,물질오용/중독,A,무의식(GCS 3-8),AC,AAAAC,1,물질오용 - 물질오용/중독 - 무의식(GCS 3-8),32
4,성인,A,물질오용,A,물질오용/중독,A,중등도 호흡곤란,AD,AAAAD,2,물질오용 - 물질오용/중독 - 중등도 호흡곤란,33
5,성인,A,물질오용,A,물질오용/중독,A,혈역학적 장애,AE,AAAAE,2,물질오용 - 물질오용/중독 - 혈역학적 장애,32


# 2. Get embeddings and save them for future reuse

In [ ]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage
import openai
openai.api_key = input()


# This may take a few minutes
df["Embedding"] = df.Combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("../data/KTAS_DATA_Embeddings.csv")